In [2]:
from stable_baselines3 import PPO
from stable_baselines3.ppo.policies import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env
import gym
import numpy as np
from RL import helpers
from Hack import load
import matplotlib.pyplot as plt
%matplotlib qt5

In [3]:
epex = load.epex().load()
price_array = epex['apx_da_hourly'].values

# Superposed Epoch Analysis

In [23]:
doy = []
years = []
for i, row in epex.iterrows():
    doy.append(i.day_of_year)
    years.append(i.year)

epex['doy'] = doy    
epex['years'] = years    

fig,axs = plt.subplots(1,1)

medians = []

for i in [2019, 2020]:
    year_df = epex.loc[(epex.year==i)]
    for doy in np.unique(year_df['doy']):
        sliced_df = year_df.loc[year_df['doy'] == doy]
        hour_through_day = np.arange(0, sliced_df.shape[0])*0.5
        
        # median_on_day = helpers.get_expected_price(sliced_df["apx_da_hourly"].values,sliced_df.shape[0], mode='median', window_size=sliced_df.shape[0])
        # medians.append(median_on_day)
        
        axs.plot(hour_through_day , sliced_df["apx_da_hourly"], color = 'blue', lw = 0.1)
        
        axs.set_ylabel("Price")
    
# for median in medians:
#     axs.axhline(median, color = 'green', lw = 0.1)
    
axs.set_xticks([0,6,12,18,24])
axs.set_xlim(0,24)
axs.grid(axis='x')
axs.set_xlabel("Hour in Day")

## Median Trend 

In [21]:
import matplotlib.pyplot as plt


fig, axs = plt.subplots(1,1, figsize = (15,10))
axs.plot(epex.index, price_array, color = 'mediumblue', label='Price')
mean_prices = []
median_prices = []
for i in np.arange(price_array.shape[0]):
    mean_prices.append(helpers.get_expected_price(price_array, i, mode = 'mean'))
    median_prices.append(helpers.get_expected_price(price_array, i, mode = 'median', window_size=2*24))
    
axs.plot(epex.index, mean_prices, color = 'cyan', label='mean')
axs.plot(epex.index, median_prices, color = 'magenta', label='median')

axs.set_ylabel("Price")
axs.legend()
fig.autofmt_xdate()